In [1]:
import sys
import glob
import numpy
import scipy
import h5py

from scipy import sparse

from dragonnfruit.io import LocusGenerator
from dragonnfruit.io import GenomewideGenerator

from dragonnfruit.models import CellStateController
from dragonnfruit.models import DynamicBPNet
from dragonnfruit.models import DragoNNFruit
from dragonnfruit.preprocessing import extract_fragments
from dragonnfruit.preprocessing import preprocess_sparse_atac
from dragonnfruit.preprocessing import read_chrom_sizes
from dragonnfruit.io import save_data, load_data


import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import muon as mu
from muon import atac as ac

import collections
from scipy.sparse import csc_matrix

In [2]:
chrom_sizes, header = read_chrom_sizes("/workspaces/torch_ddsm/_data_pool1/general/hg38.chrom.sizes")
canonical_chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 
		'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 
		'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 
		'chr22', 'chrX']

filtered_chrom_sizes = {key: value for key, value in chrom_sizes.items() if key in canonical_chroms}

chroms = list(filtered_chrom_sizes.keys())

parameters = {
	'fragments': ["/workspaces/torch_ddsm/_data_pool1/10x_data/pbmc3kv2/pbmc_granulocyte_sorted_3k_atac_fragments.tsv.gz"],
	'sequences': "/workspaces/torch_ddsm/_data_pool1/cellranger_references/refdata-gex-GRCh38-2020-A/fasta/genome.fa",
	'chrom_sizes': filtered_chrom_sizes,
	'chroms': chroms,
	'include_cells': None,
	'exclude_cells': None,
	'cell_name_prefixes': '',
	'max_fragment_length': 1000,
	'start_offset': 4,
	'end_offset': -5,
	'n_jobs': 16,
	'verbose': True,
	'signals':'dragonnfruit_data.h5',
	'read_depths':'atac_read_depths.npz',
	'save_dir': '/workspaces/torch_ddsm/_data_pool1/test_dragonnfruit_pbmc',
    'loci': '/workspaces/torch_ddsm/_data_pool1/10x_data/pbmc3kv2/pbmc_granulocyte_sorted_3k_atac_peaks.bed',
    'n_neighbors': 500,
	'n_components': 50,
 	'neighbors': 'atac_neighbors.npz',
  	'pca': 'atac_pca.npz',
	'count_matrix': 'atac_cellxpeak_counts.npz',
	'name' : "pbmc_granulocyte_sorted_3k_atac",
}
X_cscs, read_depths = extract_fragments(
		fragments=parameters['fragments'],
		chrom_sizes=parameters['chrom_sizes'],
		chroms=parameters['chroms'],
		include_cells=parameters['include_cells'],
		exclude_cells=parameters['exclude_cells'],
		cell_name_prefixes=parameters['cell_name_prefixes'],
		max_fragment_length=parameters['max_fragment_length'],
		start_offset=parameters['start_offset'],
		end_offset=parameters['end_offset'],
		n_jobs=parameters['n_jobs'],
		verbose=parameters['verbose']
)

os.makedirs(parameters['save_dir'], exist_ok=True)
save_data(f"{parameters['save_dir']}/{parameters['signals']}", X_cscs)
numpy.savez_compressed(f"{parameters['save_dir']}/{parameters['read_depths']}", read_depths)

44110005it [02:07, 345695.72it/s]



/workspaces/torch_ddsm/_data_pool1/10x_data/pbmc3kv2/pbmc_granulocyte_sorted_3k_atac_fragments.tsv.gz
Cell included:  461847
Cells excluded:  0
Cells not included:  0
Cells added:  461847
Cell inclusion list size:  0
Cell exclusion list size:  0
Frag. included count:  43986816
Frag. filtered by chrom:  16397
Frag. filtered by length:  106741



In [3]:
cellxlocus, X_pca, neighbors = preprocess_sparse_atac(
    X_cscs, 
    peaks=parameters['loci'], 
    chroms=parameters['chroms'],
    n_components=parameters['n_components'], 
    n_neighbors=parameters['n_neighbors'],
    verbose=parameters['verbose'])

scipy.sparse.save_npz(f"{parameters['save_dir']}/{parameters['count_matrix']}", cellxlocus)
numpy.savez_compressed(f"{parameters['save_dir']}/{parameters['pca']}", X_pca)
numpy.savez_compressed(f"{parameters['save_dir']}/{parameters['neighbors']}", neighbors)

0it [00:00, ?it/s]

98319it [00:08, 11377.58it/s]


### chrom pseudobulk

In [4]:
from dragonnfruit.preprocessing import create_pseudobulks
create_pseudobulks(
    reads=X_cscs, 
    output_filename=f"{parameters['save_dir']}/pbulk.bw", 
    chrom_sizes=parameters['chrom_sizes'],
    chroms=parameters['chroms'])

In [5]:
X_cscs['chr1']

<461847x248956422 sparse matrix of type '<class 'numpy.int8'>'
	with 8245994 stored elements in Compressed Sparse Column format>

### Fit bias model

##### Does this need to be cell-wise bias?

In [6]:
from bpnetlite.negatives import extract_matching_loci

bp_parameters = {
	'peaks': '/workspaces/torch_ddsm/_data_pool1/10x_data/pbmc3kv2/pbmc_granulocyte_sorted_3k_atac_peaks.bed',
	'fasta': parameters['sequences'],
	'bin_width': 0.02,
	'max_n_perc': 0.1,
	'bigwig': '/workspaces/torch_ddsm/_data_pool1/test_dragonnfruit_pbmc/pbulk.bw',
	'beta': 0.5,
	'in_window': 2114,
	'out_window': 1000,
	'verbose': True,
    'chroms': chroms,
    'max_jitter': 128,
    'output': 'matched_loci.bed',
}

# Extract regions that match the GC content of the peaks
matched_loci = extract_matching_loci(
    loci=parameters['loci'], 
    fasta=bp_parameters['fasta'],
    gc_bin_width=bp_parameters['bin_width'],
    max_n_perc=bp_parameters['max_n_perc'],
    bigwig=bp_parameters['bigwig'],
    signal_beta=bp_parameters['beta'],
    in_window=bp_parameters['in_window'],
    out_window=bp_parameters['out_window'],
    verbose=bp_parameters['verbose']
)

matched_loci.to_csv(f"{parameters['save_dir']}/{bp_parameters['output']}", header=False, sep='\t', index=False)

Loading Loci:   0%|          | 166/98319 [00:00<00:59, 1654.05it/s]

Loading Loci: 100%|██████████| 98319/98319 [01:04<00:00, 1522.82it/s]


100%|██████████| 33/33 [00:00<00:00, 16678.16it/s]


GC Bin	Background Count	Peak Count	Chosen Count
0.00:        0	       0	       0
0.02:        0	       0	       0
0.04:        0	       0	       0
0.06:        6	       0	       0
0.08:       10	       0	       0
0.10:       12	       0	       0
0.12:       17	       0	       0
0.14:       27	       0	       0
0.16:       39	       0	       0
0.18:       36	       0	       0
0.20:       70	       0	       0
0.22:      124	       4	       4
0.24:      236	       7	       7
0.26:     1362	      12	      12
0.28:     8227	      56	      56
0.30:    30605	     275	     275
0.32:    70229	    1004	    1004
0.34:   116803	    2629	    2629
0.36:   157100	    4986	    4986
0.38:   179921	    7274	    7274
0.40:   153826	    9064	    9064
0.42:   111868	   10147	   10147
0.44:    83789	    9823	    9823
0.46:    60757	    9025	    9025
0.48:    38721	    7931	    7931
0.50:    23670	    7149	    7149
0.52:    14046	    5870	    6010
0.54:     8599	    5069	    8599
0.56:     5702	    4221	    

Loading Loci: 100%|██████████| 98319/98319 [01:27<00:00, 1128.61it/s]


GC-bin KS test stat:0.0524, p-value 1.03e-117
Peak Robust Signal Minimum: 33.0
Matched Signal Maximum: 16.0


#### Train the bias model

In [9]:
# default bias parameters
from bpnetlite.io import PeakGenerator
from bpnetlite.io import extract_loci


training_data = PeakGenerator(
    loci=matched_loci, 
    sequences=parameters['sequences'],
    signals=[f"{parameters['save_dir']}/pbulk.bw"],
    controls=None,
    chroms=chroms[2:],
    in_window=bp_parameters['in_window'],
    out_window=bp_parameters['out_window'],
    max_jitter=bp_parameters['max_jitter'],
    reverse_complement=True,
    min_counts=None,
    max_counts=None,
    random_state=None,
    batch_size=64,
    verbose=True
)

valid_data = extract_loci(
    sequences=parameters['sequences'],
    signals=[f"{parameters['save_dir']}/pbulk.bw"],
    controls=None,
    loci=matched_loci,
    chroms=chroms[:2],
    in_window=bp_parameters['in_window'],
    out_window=bp_parameters['out_window'],
    max_jitter=bp_parameters['max_jitter'],
    verbose=True,
)

valid_sequences, valid_signals = valid_data
valid_controls = None
n_control_tracks = 0

Loading Loci:   0%|          | 0/82087 [00:00<?, ?it/s]

Loading Loci: 100%|██████████| 16232/16232 [00:12<00:00, 1315.10it/s]


In [10]:
bias_fit_parameters = {
    "n_filters": 256,
    "n_layers": 4,
    "max_counts": None,
    "loci": "../../../chromatin-atlas/ATAC/ENCSR637XSC/negatives_data/negatives.bed",
    "verbose": True,
    "random_state": None,
	"alpha": 10,
	"beta": 0.5,
    "lr": 0.000001,
    "max_epochs": 10,
    "batch_size": 512,
    "n_control_tracks": n_control_tracks,
    "valid_controls": valid_controls,
    "n_outputs" : 1,
    "validation_iter": 1000,
}

# bias_fit_parameters = parameters.copy()

name = f"{parameters['name']}.chrombpnet.bias.fit.json"
bias_fit_parameters['name'] = f"{parameters['name']}.bias"
parameters['bias_model'] = f"{bias_fit_parameters['name']}.torch"

min_counts = training_data.dataset.signals.sum(dim=(1, 2)).min().item()
bias_fit_parameters['max_counts'] = min_counts * bias_fit_parameters['beta']

#### Run BPnet fit

In [11]:
import torch

from bpnetlite.bpnet import BPNet

trimming = (bp_parameters['in_window'] - bp_parameters['out_window']) // 2

model = BPNet(n_filters=bias_fit_parameters['n_filters'], 
    n_layers=bias_fit_parameters['n_layers'],
    profile_output_bias=True,
    count_output_bias=True,
    alpha=bias_fit_parameters['alpha'],
    trimming=trimming,
    name=f"{parameters['save_dir']}/{parameters['name']}",
    n_control_tracks=bias_fit_parameters['n_control_tracks'],
    n_outputs=bias_fit_parameters['n_outputs']).cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr=bias_fit_parameters['lr'])

model.fit(training_data, optimizer, X_valid=valid_sequences, 
		X_ctl_valid=valid_controls, y_valid=valid_signals, max_epochs=bias_fit_parameters['max_epochs'], 
    batch_size=bias_fit_parameters['batch_size'],validation_iter=bias_fit_parameters['validation_iter'],verbose=bias_fit_parameters['verbose'])


Epoch	Iteration	Training Time	Validation Time	Training MNLL	Training Count MSE	Validation MNLL	Validation Profile Pearson	Validation Count Pearson	Validation Count MSE	Saved?
0	0	1.299	3.2782	37.5275	4.6096	45.9182	0.0011741763	0.12038599	5.3281	True
0	1000	36.343	2.7568	33.5983	0.5789	44.6027	0.076191746	-0.091623284	0.8147	True
Epoch 0: Average Validation Loss = 88.95395052367974
1	2000	23.7664	2.7651	35.3002	0.6034	43.1947	0.097735494	0.11622072	0.8001	True
Epoch 1: Average Validation Loss = 52.064289390343944
2	3000	14.4085	2.7706	33.318	0.7486	41.7538	0.107549556	0.1784533	0.7775	True
Epoch 2: Average Validation Loss = 50.09292374192414
3	4000	5.0294	2.7607	33.2003	0.8144	40.3799	0.11340245	0.18840684	0.783	True
3	5000	35.8815	2.7702	27.0193	0.7897	39.4532	0.11583789	0.1907709	0.7764	True
Epoch 3: Average Validation Loss = 48.262766575683216
4	6000	28.7576	2.7705	31.039	0.7894	38.9984	0.116843775	0.19193283	0.766	True
Epoch 4: Average Validation Loss = 47.03655629741523
5	7000	19.

## Run attribution/motif diagnosis on the bias model

#### Attribution params

In [57]:
attribute_parameters = {
	'sequences': parameters['sequences'],
	'loci': parameters['loci'],
 	'batch_size': 5,
	'in_window': 2114,
	'out_window': 1000,
	'verbose': True,
	'max_jitter': 0,
	'chroms': ['chr6', 'chr7', 'chrX'],
	'model': None,
	'output': 'profile',
	'n_shuffles':20,
	'random_state':0,
 	'ohe_filename': f"{parameters['save_dir']}/{parameters['name']}.ohe.npz",
	'attr_filename': f"{parameters['save_dir']}/{parameters['name']}.attr.npz",
}

#### Run attribution on the bias model

In [49]:
attribute_parameters['sequences']

In [54]:
from bpnetlite.attributions import attribute

model = torch.load(f"{parameters['save_dir']}/{parameters['name']}.final.torch").cuda()

X = extract_loci(
    sequences=attribute_parameters['sequences'],
    loci=attribute_parameters['loci'],
    chroms=attribute_parameters['chroms'],
    max_jitter=attribute_parameters['max_jitter'],
    verbose=attribute_parameters['verbose']
)

Loading Loci:   0%|          | 0/13711 [00:00<?, ?it/s]

Loading Loci: 100%|██████████| 13711/13711 [00:26<00:00, 517.29it/s]


In [58]:
X_ctl = None
X_attr = attribute(model, X, args=X_ctl,
    model_output=attribute_parameters['output'], hypothetical=True, 
    n_shuffles=attribute_parameters['n_shuffles'],
    batch_size=attribute_parameters['batch_size'],
    random_state=attribute_parameters['random_state'],
    verbose=attribute_parameters['verbose'])

numpy.savez_compressed(attribute_parameters['ohe_filename'], X)
numpy.savez_compressed(attribute_parameters['attr_filename'], X_attr)

100%|██████████| 2743/2743 [05:54<00:00,  7.73it/s]


#### Modisco params

In [71]:
modisco_motifs_parameters = {
    'n_seqlets': 100000,
    'output_filename': f"{parameters['save_dir']}/{parameters['name']}_modisco_results.h5",
    'verbose': True,
    'n_leiden': 2,
    'window': 400,
}

#### Run modisco on the attributions

In [73]:
import modiscolite
from modiscolite.util import calculate_window_offsets

sequences = np.load(attribute_parameters['ohe_filename'])['arr_0']
attributions = np.load(attribute_parameters['attr_filename'])['arr_0']

center = sequences.shape[2] // 2
start, end = calculate_window_offsets(center, modisco_motifs_parameters['window'])

sequences = sequences[:, :, start:end].transpose(0, 2, 1)
attributions = attributions[:, :, start:end].transpose(0, 2, 1)

if sequences.shape[1] < modisco_motifs_parameters['window']:
    raise ValueError("Window ({}) cannot be ".format(modisco_motifs_parameters['window']) +
        "longer than the sequences".format(sequences.shape))

sequences = sequences.astype('float32')
attributions = attributions.astype('float32')

pos_patterns, neg_patterns = modiscolite.tfmodisco.TFMoDISco(
    hypothetical_contribs=attributions, 
    one_hot=sequences,
    max_seqlets_per_metacluster=modisco_motifs_parameters['n_seqlets'],
    sliding_window_size=20,
    flank_size=5,
    target_seqlet_fdr=0.05,
    n_leiden_runs=modisco_motifs_parameters['n_leiden'],
    verbose=modisco_motifs_parameters['verbose'])

modiscolite.io.save_hdf5(modisco_motifs_parameters['output_filename'], pos_patterns, neg_patterns, modisco_motifs_parameters['window'])

Using 11910 positive seqlets
Extracted 2033 negative seqlets


#### Parameterize the modisco report

In [77]:
modisco_motifs_parameters['output_filename']

'/workspaces/torch_ddsm/_data_pool1/test_dragonnfruit_pbmc/pbmc_granulocyte_sorted_3k_atac_modisco_results.h5'

In [78]:
modisco_report_parameters= {
    'motifs': '/workspaces/torch_ddsm/_data_pool1/jaspar/JASPAR2020_CORE_vertebrates_non-redundant_pfms_meme.txt',
    'output_folder': f"{parameters['save_dir']}/{parameters['name']}_modisco/",
    'verbose': True,
    'input': modisco_motifs_parameters['output_filename'],
    
}

#### Generate the modisco report

In [79]:
# Step 5: Generate the tf-modisco report
modiscolite.report.report_motifs(
    modisco_report_parameters['input'], 
    modisco_report_parameters['output_folder'],
    img_path_suffix=modisco_report_parameters['output_folder'], 
    meme_motif_db=modisco_report_parameters['motifs'],
    is_writing_tomtom_matrix=True,
    top_n_matches=3)

In [80]:
from IPython.display import HTML

# Open the HTML file and read its content
with open(
    '/workspaces/torch_ddsm/_data_pool1/test_dragonnfruit_pbmc/pbmc_granulocyte_sorted_3k_atac_modisco/motifs.html', 
    'r') as file:
    html_content = file.read()

# Display the HTML content
HTML(html_content)

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,3832,,,MA0146.2,1.000000,,MA1600.1,1.000000,,MA1619.1,1.000000,
pos_patterns.pattern_1,600,,,MA0836.2,1.000000,,MA0102.4,1.000000,,MA0800.1,1.000000,
pos_patterns.pattern_2,583,,,MA0003.4,1.000000,,NaN,NaN,,NaN,NaN,
pos_patterns.pattern_3,539,,,MA0116.1,1.000000,,MA0750.2,1.000000,,MA1619.1,1.000000,
pos_patterns.pattern_4,499,,,MA1619.1,1.000000,,MA1615.1,1.000000,,MA0017.2,1.000000,
pos_patterns.pattern_5,308,,,MA0163.1,1.000000,,MA0017.2,1.000000,,MA0116.1,1.000000,
pos_patterns.pattern_6,307,,,MA0163.1,1.000000,,MA0146.2,1.000000,,MA0809.2,1.000000,
pos_patterns.pattern_7,237,,,MA0812.1,0.999999,,MA0003.4,0.999999,,NaN,NaN,
pos_patterns.pattern_8,182,,,MA0146.2,0.471498,,MA1615.1,0.471498,,MA1643.1,0.471498,
pos_patterns.pattern_9,153,,,MA0781.1,1.000000,,MA0779.1,1.000000,,MA0138.2,1.000000,
